In [53]:
import numpy as np
import scipy.io
from scipy.linalg import lstsq
import sounddevice as sd
import matplotlib.pyplot as plt
import helper_func as F
import os
import scipy.io.wavfile as wav
import sys

sys.path.append(os.path.abspath('data'))
import load_data

In [54]:
# OMP Algorithm
# takes measurements y, measurement
# ouputs signal x with s atom indices

# adpat OMP for image processing

def OMP(A, y, stop, tolerance = 1e-6):                  # max iterations should be n
    m, n = A.shape
    y = np.asarray(y).ravel()
    r = y                                               # r_0 = s
    lambdas = []                                        # support
    x_k = np.zeros(n)

    i = 0
    while(i < stop):                                    # stop condition: max iterations
        correlation = np.abs(A.T @ r)                   # correlation of each atom with current residual

        lambda_k = np.argmax(correlation)               # ith best atom (max correlation)
        lambdas.append(lambda_k)

        A_sel = A[:, lambdas]                           # basis of atoms matrix
        ls = F.leastSquare(A_sel, y)
        x_k[lambdas] = ls

        r = y - (A_sel @ ls)                            # calc residual

        if F.l2Norm(r) < tolerance:                     
            break

        i += 1

    return x_k, lambdas

In [59]:
y, D, A = load_data.loading_data('data/')

In [56]:
fps = 7350
int16_max = np.iinfo(np.int16).max
wav.write('audio_results/compressed_y.wav', fps, (y * int16_max).astype(np.int16))
sd.play(y, fps)

$\Theta = AD$. Then $y = \Theta s$.

In [57]:
K_list = [10, 50, 100, 200, 300, 1000, 2000, 3000]
results = {}
s = 100
t = A @ D                           # effective sensing matrix theta

os.makedirs("audio_results", exist_ok=True)

for k in K_list:
    y_k = y[:k].ravel()
    tk = t[:k, :]                   # theta

    x_k, lambdas = OMP(tk, y_k, s)     # recover x_k and lambdas with OMP of sparsity 100
    x_s = D @ x_k

    sd.play(x_s, fps)
    sd.wait()
    wav.write(f'audio_results/recovered_OMP_K{k}.wav', fps, (x_s * int16_max).astype(np.int16))
    results[k] = x_s


In [58]:

s_ls = F.leastSquare(t, y.ravel())
x_ls = D @ s_ls
peak_ls = np.max(np.abs(x_ls))
sd.play(x_ls, fps)

wav.write('audio_results/recovered_LS_K3000.wav', fps, (x_ls * int16_max).astype(np.int16))


